In [1]:
from processData import MLDataAnalyzer
import pandas as pd
from generateConfig import GenerateConfigLLM
from trainingSchool import TrainingSchool 

2025-01-12 11:33:55.275270: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-12 11:33:55.287957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736661835.303521  369197 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736661835.307845  369197 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 11:33:55.323872: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [15]:
df = pd.read_csv("example_data.csv")

In [16]:


task_description = "Using the dataframe I need a binary classification algorithm. This is a sequence data"

models = [
    'StandardScaler',
    'MinMaxScaler',
    'RobustScaler',
    'LinearRegression',
    'Ridge',
    'RandomForestRegressor',
    'SVR',
    'XGBRegressor',
    'MLPRegressor',
    'LogisticRegression',
    'RandomForestClassifier',
    'SVC',
    'XGBClassifier',
    'MLPClassifier'
]


analyzer = MLDataAnalyzer(df, "target", task_description)
sample_data = analyzer.get_sample_data()
analysis = analyzer.analyze_dataset()
config_generator = GenerateConfigLLM()
config = config_generator.generate_config(analysis, sample_data, task_description, models)
print(config)

```yml
scalers:
  standard_scaler:
    name: StandardScaler
    params: {}

preprocessing:
  missing_values:
    strategy: None
  categorical:
    strategy: None
  feature_selection:
    enabled: False
    method: None
    params: {}

models:
  classification:
    logistic_regression:
      name: LogisticRegression
      params: {}
    random_forest_classifier:
      name: RandomForestClassifier
      params: {}

deep_learning:
  lstm:
    enabled: True
    params:
      units: [64, 32]
      dropout_rates: [0.2, 0.1]
      optimizer: Adam
      batch_size: 32
      epochs: 10
  transformer:
    enabled: True
    params:
      num_heads: 2
      key_dim: 32
      dropout_rate: 0.1
      dense_units: [64, 32]
      optimizer: Adam
      batch_size: 32
      epochs: 10

evaluation:
  cv_folds: 5
  stratify: False
  metrics: 
    classification: ['accuracy', 'f1', 'precision', 'recall']

sequence_data:
  enabled: True
```


In [17]:
import pickle
import os

def save_model(model, filepath):
    """
    Save a trained model to a file using pickle.
    
    Parameters:
    model: sklearn model object
        The trained model to save
    filepath: str
        Path where the model will be saved
    """
    try:
        with open(filepath, 'wb') as file:
            pickle.dump(model, file)
        print(f"Model successfully saved to {filepath}")
    except Exception as e:
        print(f"Error saving model: {str(e)}")

def load_model(filepath):
    """
    Load a model from a pickle file.
    
    Parameters:
    filepath: str
        Path to the saved model file
    
    Returns:
    The loaded model object
    """
    try:
        with open(filepath, 'rb') as file:
            model = pickle.load(file)
        print(f"Model successfully loaded from {filepath}")
        return model
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        return None

In [18]:
yaml_content = config.strip().strip('`').lstrip('yml').strip()


In [19]:
 trainer = TrainingSchool(config_path="config1.yaml")

# Assuming X and y are your data
# For demonstration, let's create dummy data:
X = df.drop("target", axis=1).values
y = df["target"].values


# Fit the model
trainer.fit(X, y)

# Retrieve and print best model details
best_info = trainer.get_best_model()
print("Best Model Info:", best_info)

No GPU found. Running on CPU.
Detected task type: classification
Score for logistic_regression with standard scaler: 0.9800
Score for LSTM with standard scaler: 0.9493
Score for Transformer with standard scaler: 0.5250

Best model: logistic_regression
Best scaler: StandardScaler
Best score: 0.9800
Best Model Info: {'model': LogisticRegression(), 'scaler': StandardScaler(), 'score': 0.9800000000000001, 'model_name': 'logistic_regression'}


In [30]:
import tempfile
import os

filename = "example.csv"

with tempfile.NamedTemporaryFile(mode='w+', suffix='.yaml', delete=False) as temp_config_file:
    temp_config_file.write(yaml_content)
    temp_config_file.flush()
    config_path = temp_config_file.name
    
    trainer = TrainingSchool(config_path=config_path)
    X = df.drop("target", axis=1).values
    y = df["target"].values
    
    trainer.fit(X, y)
    best_info = trainer.get_best_model()
    
    # Save the actual model using joblib
    model_filename = f"{filename.rsplit('.', 1)[0]}_model.pkl"

    test_df = df.drop("target", axis=1)

    model_info = {
        'filename': filename,
        'task_description': task_description,
        'best_model': {
            'model_type': type(best_info['model']).__name__,
            'score': best_info.get('score', None),
            'model_path': model_filename  # Save the path to the saved model
        },
        'feature_columns': list(df.drop("target", axis=1).columns)
    }


    required_features = model_info['feature_columns']
    if not all(feature in test_df.columns for feature in required_features):
        print(jsonify({'error': 'Test data missing required features'}))
    




    save_model(best_info['model'], model_filename)
    
    loaded_model = load_model(model_filename)

    X_test_scaled = best_info['scaler'].transform(X_test)
    predictions = best_info['model'].predict(X_test_scaled)


    # Create a new DataFrame with predictions
    result_df = test_df.copy()
    result_df['target'] = predictions
    y_test = loaded_model.predict(X_test_scaled)




    
    # Create serializable model info
   
    
    print(model_info)
    
    os.unlink(config_path)
    
  

No GPU found. Running on CPU.
Detected task type: classification
Score for logistic_regression with standard_scaler scaler: 0.9800
Score for random_forest_classifier with standard_scaler scaler: 0.8750
Score for LSTM with standard_scaler scaler: 0.8625
Score for Transformer with standard_scaler scaler: 0.5250

Best model: logistic_regression
Best scaler: StandardScaler
Best score: 0.9800
Model successfully saved to example_model.pkl
Model successfully loaded from example_model.pkl
{'filename': 'example.csv', 'task_description': 'Using the dataframe I need a binary classification algorithm. This is a sequence data', 'best_model': {'model_type': 'LogisticRegression', 'score': 0.9800000000000001, 'model_path': 'example_model.pkl'}, 'feature_columns': ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4']}


In [31]:


test_df = df.drop("target", axis=1)

# X.to_csv("test_data.csv", index=False)

In [32]:



model_path = "example_model.pkl"
model = load_model(model_path)
# Make predictions using the model


Model successfully loaded from example_model.pkl


In [33]:
result_df

,feature_0,feature_1,feature_2,feature_3,feature_4,target
0,0.374540,0.950714,0.731994,0.598658,0.156019,1
1,0.155995,0.058084,0.866176,0.601115,0.708073,0
2,0.020584,0.969910,0.832443,0.212339,0.181825,0
3,0.183405,0.304242,0.524756,0.431945,0.291229,0
4,0.611853,0.139494,0.292145,0.366362,0.456070,0
...,...,...,...,...,...,...
195,0.262264,0.595078,0.051426,0.496366,0.596843,0
196,0.334244,0.770912,0.106598,0.075138,0.728189,0
197,0.495491,0.688402,0.434827,0.246402,0.819102,1
198,0.799416,0.694696,0.272145,0.590231,0.360974,1


In [34]:
result_df.target.unique()

array([1, 0])

In [39]:
from flask import send_file

result_df = test_df.copy()
result_df['target'] = predictions

# 7. Create a temporary file to store the CSV
temp_file = tempfile.NamedTemporaryFile(mode='w', suffix='.csv', delete=False, newline='')
result_df.to_csv(temp_file.name, index=False)
temp_file_path = temp_file.name
temp_file.close()

model_name = "LogisticRegression"

# 9. Generate output filename
output_filename = f"predictions_1.csv"

send_file(
    temp_file_path,
    mimetype='text/csv',
    as_attachment=True,
    download_name=output_filename
)

RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.

In [1]:
from main import IterativeModelTrainer
import pandas as pd

df = pd.read_csv('example_data.csv')

# Initialize trainer
trainer = IterativeModelTrainer(
    df=df,
    task_description="Your task description",
    max_iterations=2
)

# Run training
iterations, best_model = trainer.train_iteratively()

# Make predictions
X_test = df.drop("target", axis=1).values
predictions = best_model['model'].predict(best_model['scaler'].transform(X_test))

# Create results DataFrame
results = pd.DataFrame(predictions, columns=['predictions'])

2025-01-12 11:35:30.823604: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-12 11:35:30.841867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736661930.859340  369989 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736661930.864389  369989 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 11:35:30.881460: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

scalers:
  standard_scaler:
    name: StandardScaler
    params: {}

preprocessing:
  missing_values:
    strategy: None
  categorical:
    strategy: None
  feature_selection:
    enabled: false
    method: None
    params: {}

models:
  regression:
    linear_regression:
      name: LinearRegression
      params: {}
    ridge_regression:
      name: Ridge
      params: {}
    random_forest_regressor:
      name: RandomForestRegressor
      params: {"n_estimators": 100, "max_depth": 5}
    xgb_regressor:
      name: XGBRegressor
      params: {"n_estimators": 100, "max_depth": 5}
    mlp_regressor:
      name: MLPRegressor
      params: {"hidden_layer_sizes": [10]}

deep_learning:
  lstm:
    enabled: false
    params: {}
  transformer:
    enabled: false
    params: {}

evaluation:
  cv_folds: 5
  stratify: false
  metrics: 
    regression: ["mean_squared_error", "mean_absolute_error", "r2_score"]

sequence_data:
  enabled: false
No GPU found. Running on CPU.
Detected task type: class

KeyError: 'classification'

In [1]:
import pandas as pd
import os
import tempfile
from processData import MLDataAnalyzer
from generateConfig import GenerateConfigLLM
from model_analysis import ModelAnalysisLLM  
from trainingSchool import TrainingSchool 
import logging
import time
from sklearn.model_selection import train_test_split
import sys
import numpy as np





class IterativeModelTrainer:
    def __init__(self, df, task_description, target_column="target", max_iterations=2):
        self.df = df
        self.task_description = task_description
        self.target_column = target_column
        self.max_iterations = max_iterations
        self.analyzer = MLDataAnalyzer(df, target_column, task_description)
        self.model_analyzer = ModelAnalysisLLM()
        self.config_generator = GenerateConfigLLM()  # Add initial config generator
        self.available_models = [
            'StandardScaler', 'MinMaxScaler', 'RobustScaler',
            'LinearRegression', 'Ridge', 'RandomForestRegressor',
            'SVR', 'XGBRegressor', 'MLPRegressor',
            'LogisticRegression', 'RandomForestClassifier',
            'SVC', 'XGBClassifier', 'MLPClassifier'
        ]
        
    def _train_and_evaluate(self, config_yaml, X_train, X_test, y_train, y_test):
        """Train models and collect performance metrics"""
        with tempfile.NamedTemporaryFile(mode='w+', suffix='.yaml', delete=False) as temp_file:
            temp_file.write(config_yaml)
            temp_file.flush()
            
            trainer = TrainingSchool(config_path=temp_file.name)
            trainer.fit(X_train, y_train)
            
            results = {}
            for model_name, model_info in trainer.models.items():
                model = model_info['model']
                scaler = model_info['scaler']
                
                # Time the training
                start_time = time.time()
                model.fit(scaler.transform(X_train), y_train)
                training_time = time.time() - start_time
                
                # Time the prediction
                start_time = time.time()
                y_pred = model.predict(scaler.transform(X_test))
                prediction_time = time.time() - start_time
                
                # Get memory usage
                memory_usage = sys.getsizeof(model) + sys.getsizeof(scaler)
                
                # Collect CV scores
                cv_scores = trainer.get_cv_scores(model_name)
                
                # Get feature importance if available
                feature_importance = None
                if hasattr(model, 'feature_importances_'):
                    feature_importance = dict(zip(
                        self.df.drop(self.target_column, axis=1).columns,
                        model.feature_importances_
                    ))
                
                results[model_name] = {
                    'y_true': y_test,
                    'y_pred': y_pred,
                    'cv_scores': cv_scores,
                    'training_time': training_time,
                    'prediction_time': prediction_time,
                    'memory_usage': memory_usage,
                    'feature_importance': feature_importance,
                    'convergence_info': {
                        'n_iter_': getattr(model, 'n_iter_', None),
                        'loss_curve_': getattr(model, 'loss_curve_', None)
                    }
                }
            
            os.unlink(temp_file.name)
            return results, trainer.get_best_model()
    
    def train_iteratively(self):
        """Perform iterative training with model analysis and refinement"""
        # Initial analysis and data split
        analysis = self.analyzer.analyze_dataset()
        sample_data = self.analyzer.get_sample_data()
        X = self.df.drop(self.target_column, axis=1).values
        y = self.df[self.target_column].values
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Storage for tracking progress
        all_iterations = []
        best_model_info = None
        best_score = float('-inf')
        
        # Get initial configuration using GenerateConfigLLM
        initial_config = self.config_generator.generate_config(
            analysis, sample_data, self.task_description, self.available_models
        )
        yaml_content = initial_config.strip().strip('`').lstrip('yml').strip()

        print(self.task_description)

        print(yaml_content)
        
        # First iteration with initial configuration
        logging.info("Starting first iteration with GenerateConfigLLM configuration")
        results, iteration_best = self._train_and_evaluate(
            yaml_content, X_train, X_test, y_train, y_test
        )

        print(results)
        

        # Generate performance report
        performance_report = self.model_analyzer.generate_performance_report(
            results,
            'classification' if len(np.unique(y)) <= 10 else 'regression'
        )
        
        # Store first iteration results
        iteration_info = {
            'iteration': 1,
            'config': yaml_content,
            'results': results,
            'best_model': iteration_best,
            'performance_report': performance_report
        }
        all_iterations.append(iteration_info)
        
        # Update best model
        best_score = iteration_best['score']
        best_model_info = iteration_best
        
        logging.info(f"Completed first iteration with score: {best_score}")
        
        # Continue with remaining iterations using ModelAnalysisLLM
        current_config = yaml_content
        for iteration in range(1, self.max_iterations):
            logging.info(f"Starting iteration {iteration + 1}/{self.max_iterations}")
            
            # Generate refined configuration
            current_config = self.model_analyzer.generate_refined_config(
                analysis,
                results,
                'classification' if len(np.unique(y)) <= 10 else 'regression',
                self.available_models
            )
            
            # Train and evaluate models
            results, iteration_best = self._train_and_evaluate(
                current_config, X_train, X_test, y_train, y_test
            )

            print("Results")

            print(results)

            return
            
            # Generate performance report
            performance_report = self.model_analyzer.generate_performance_report(
                results,
                'classification' if len(np.unique(y)) <= 10 else 'regression'
            )
            
            # Store iteration results
            iteration_info = {
                'iteration': iteration + 1,
                'config': current_config,
                'results': results,
                'best_model': iteration_best,
                'performance_report': performance_report
            }
            all_iterations.append(iteration_info)
            
            # Update best model if better
            if iteration_best['score'] > best_score:
                best_score = iteration_best['score']
                best_model_info = iteration_best
            
            logging.info(f"Completed iteration {iteration + 1} with best score: {best_score}")
        
        return all_iterations, best_model_info

2025-01-12 12:27:34.447279: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-12 12:27:34.458194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736665054.472838  395974 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736665054.477075  395974 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 12:27:34.491509: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
df = pd.read_csv("example_data.csv")

task_description = "Using the dataframe I need a binary classification algorithm"

trainer = IterativeModelTrainer(
    df=df,
    task_description=task_description,
    max_iterations=2  
)

# Run training
iterations, best_model = trainer.train_iteratively()

Using the dataframe I need a binary classification algorithm
scalers:
  standard_scaler:
    name: StandardScaler
    params: {}

preprocessing:
  missing_values:
    strategy: None
  categorical:
    strategy: None
  feature_selection:
    enabled: false
    method: None
    params: {}

models:
  classification:
    logistic_regression:
      name: LogisticRegression
      params:
        penalty: 'l2'
        C: 1.0
        random_state: 42
    random_forest:
      name: RandomForestClassifier
      params:
        n_estimators: 100
        max_depth: 5
        random_state: 42
    xgb_classifier:
      name: XGBClassifier
      params:
        n_estimators: 100
        max_depth: 5
        learning_rate: 0.1
        random_state: 42

deep_learning:
  lstm:
    enabled: false
    params:
      units: []
      dropout_rates: []
      optimizer: None
      batch_size: 32
      epochs: 10
  transformer:
    enabled: false
    params:
      num_heads: 8
      key_dim: 64
      dropout_ra

AttributeError: 'str' object has no attribute 'model_name'